In [5]:
import pickle
import sys
sys.path.insert(0,'../code/')
from shared import *
import pandas as pd

In [7]:
K = 5
F = 5
results = defaultdict(dict)
features = defaultdict(dict)
for k in range(2, K+1):
    with open(f'../results/milp/rawresults/result_{k}{F}.pickle','rb') as p:
        ti,(tc,tb) = pickle.load(p)
    for f in range(2, F+1):
            results[k][f] = (tc[f-2],tb[f-2])
    for f in range(2,F+1):
             features[k][f] = ti[f-2]

tracts = features[2][2].index.values.tolist()

In [13]:
average_values = {}
piecewise_data = pd.read_csv(filepath+"NY_2000_NEW.csv")
# calculate MI with respect ot the label and rank them
output = piecewise_data.set_index('trtid')
miScoredf = getMI(output, 'incpc')
miScoredf = miScoredf.sort_values(by=['MI'], ascending=False)
# number of features to keep (by highest MI)
keep_feature = miScoredf[:5].feature.values
for k,fs in results.items():
    average_values[k] = {}
    for f, (rs,_) in fs.items():
        average_values[k][f] = {}
        for m in range(1,k+1):
            model = rs['Cluster' + str(m)].values
            average_values[k][f][m] = dict(zip(keep_feature,model))


In [14]:
av = pd.DataFrame()
av['k'] = 0
av['f'] = 0
for k in average_values[5][5][1].keys():
    av[k] = ''
n = []
for k, fs in average_values.items():
    for f, ms in fs.items():
        assignments = defaultdict(list)
        for v,tk in (zip(
            features[
                k][
                f].index.values.tolist(),
            features[k
                   ][
                f]['model'].values.tolist())):
            assignments[tk].append(v)
        for m, cs in ms.items():
            t = {
                'k':k,
                'f':f,
            }
            for c, v in cs.items():
                t[c] = v
            for col in av.columns:
                if col not in t.keys():
                    t[col] = 'N/A'
            t['assignments'] = assignments[m]
            t['model'] = m
            n.append(t)
av = pd.DataFrame(n)
print(av.columns)
av = av[['k','f','model','prof', 'col', 'flabf', 'multi', 'own','assignments']]
av.to_csv('/home/alex/Desktop/milp_weight_averages.csv')

Index(['assignments', 'col', 'f', 'flabf', 'k', 'model', 'multi', 'own',
       'prof'],
      dtype='object')


In [15]:
av = av[['k','f','model','prof', 'col', 'flabf', 'multi', 'own','assignments']]
av.to_csv('/home/alex/Desktop/milp_weight_averages.csv')
av[av.k == 5][av.f == 5]

/home/alex/anaconda3/envs/isabelle/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,k,f,model,prof,col,flabf,multi,own,assignments
51,5,5,1,0.519719,0.190684,0,0,0,"[36061000201, 36061000800, 36061001402, 360610..."
52,5,5,2,0.110296,0.000000,0,0.778089,2.03945,"[36061000202, 36061001200, 36061001501, 360610..."
53,5,5,3,0.734264,1.472848,0,0.301736,0.985028,"[36061001002, 36061001600, 36061002500, 360610..."
54,5,5,4,0.298900,0.000000,0.271225,0.317434,0.313829,"[36061002201, 36061002202, 36061002800, 360610..."
55,5,5,5,0.000000,0.000000,0,0,0,"[36061000600, 36061002000, 36061003200, 360610..."


In [20]:
bv = []
features = ['prof', 'col', 'flabf', 'multi', 'own','incpc']
for i in range(len(av)):
    t = {}
    k = av.iat[i,0]
    f = int(av.iat[i,1])
    d = readindata_std('incpc',f)
    model = av.iat[i,2]
    assignments = av.iat[i,8]
    fdict = defaultdict(list)
    for asmt in assignments:
        for feat in features[0:f]:
            fdict[feat].append(
                d[d.trtid == asmt][feat].values[0]
            )
        fdict['incpc'].append(
            d[d.trtid == asmt]['incpc'].values[0]
        )
    for feat in features:
        t[feat] = np.mean(fdict[feat])
    t['k'] = k
    t['f'] = f
    t['model'] = av.iat[i,2]
    bv.append(t)
bv = pd.DataFrame(bv)

In [21]:
bv = bv[['k','f','model','prof', 'col', 'flabf', 'multi', 'own','incpc']]
bv.to_csv('/home/alex/Desktop/milp_feature_averages.csv')

In [22]:
bv[bv.k == 2][bv.f == 5]

/home/alex/anaconda3/envs/isabelle/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,k,f,model,prof,col,flabf,multi,own,incpc
6,2,5,1,-0.024022,-0.006771,0.063478,0.014142,0.008735,0.039378
7,2,5,2,0.017663,0.004979,-0.046675,-0.010398,-0.006423,-0.028955
